In [33]:
"""
    https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html
        - AG_NEWS 데이터셋으로 Text를 분류하는 모델 학습
"""

<class 'torchtext.data.datasets_utils._RawTextIterableDataset'>
vocab size = 95811


In [6]:
"""
    학습을 위한 데이터를 준비하는 과정
       - Iterable Dataset 불러옴 (여기서 사용하는 AG_NEWS는 torch.utils.dataset과 다름)
       - 불러온 Data 기반으로Vocab 만듬
"""

import torch
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# AG New 데이터 불러옴
# https://pytorch.org/text/stable/datasets.html#ag-news
# train: 120000 / test: 7600 / class 수 = 4
train_iter = AG_NEWS(root="./data/",split='train') 
print(type(train_iter))
# next(train_iter) # (class, 텍스트)
tokenizer = get_tokenizer('basic_english')

# generator - text를 tokenization하여 yield
def yield_tokens(data_iter):
    for _,text in data_iter:
        yield tokenizer(text)



# 전체 text를 tokenization 하며 vocab 구축 
vocab = build_vocab_from_iterator(yield_tokens(train_iter),specials=["<unk>"]) 
vocab.set_default_index(vocab["<unk>"])

vocab_list = vocab.get_itos()
print("vocab size = %d"%len(vocab_list))

print(vocab(["here","is"]))

<class 'torchtext.data.datasets_utils._RawTextIterableDataset'>
vocab size = 95811
[475, 21]


In [24]:
"""
    IterableDataset을 활용! -> user가 정의한 Iterator에 따라 data loading 순서가 정해짐
    DataLoader를 활용하여 학습을 위한 Batch 데이터 생성
"""
from torch.utils.data import DataLoader

train_iter = AG_NEWS(root="./data/",split='train') 

get_token_id_list = lambda x : vocab(tokenizer(x))
get_label_id = lambda x : x-1


def collate_batch(batch):
    """
        dataset을 전처리 하여 
        batch 형태의 training_example, label, offset을 Tensor로 만듬
        
        Arguments:
            batch (List(Tuple)) - (label,text) - dataset으로부터 얻어진 tuple들의 묶음
        Return:
            아래의 Return은 Cutomized collate_fn의 return 값으로 향후 dataloader로부터 iteration 마다 추출할 batch 단위의 값들임 
            
            text_list (1d Tensor) - [batch_size * (each token num)] batch를 구성하는 각 Token들의 id list
            label_list (1d Tensor) - [batch size] batch를 구성하는 label들의 id list
            offset (1d Tensor) - [batch size] text_list에서 batch를 구성하는 각 example들의 index들    
    """

    label_list, text_list, offsets = [], [],[0]
    for (_label, _text) in batch:
        label_list.append(get_label_id(_label))
        text_tensor = torch.tensor(get_token_id_list(_text),dtype=torch.int64)
        text_list.append(text_tensor)
        offsets.append(text_tensor.size(0)) 
    
    offsets=torch.tensor(offsets).cumsum(dim=0)
    label_list = torch.tensor(label_list) # [batch size * 1(label id)]
    text_list = torch.cat(text_list,dim=0) # [batch_size * (each token num)]
    
    return label_list, text_list, offsets
        
        
# print(next(train_iter))
dataloader=DataLoader(train_iter,batch_size=8,shuffle=False,collate_fn=collate_batch)
# dataloader=DataLoader(train_iter,batch_size=1,shuffle=False)

print(next(iter(dataloader)))

(tensor([2, 2, 2, 2, 2, 2, 2, 2]), tensor([  431,   425,     1,  1605, 14838,   113,    66,     2,   848,    13,
           27,    14,    27,    15, 50725,     3,   431,   374,    16,     9,
        67507,     6, 52258,     3,    42,  4009,   783,   325,     1, 15874,
         1072,   854,  1310,  4250,    13,    27,    14,    27,    15,   929,
          797,   320, 15874,    98,     3, 27657,    28,     5,  4459,    11,
          564, 52790,     8, 80617,  2125,     7,     2,   525,   241,     3,
           28,  3890, 82814,  6574,    10,   206,   359,     6,     2,   126,
            1,    58,     8,   347,  4582,   151,    16,   738,    13,    27,
           14,    27,    15,  2384,   452,    92,  2059, 27360,     2,   347,
            8,     2,   738,    11,   271,    42,   240, 51953,    38,     2,
          294,   126,   112,    85,   220,     2,  7856,     6, 40066, 15380,
            1,    70,  7376,    58,  1810,    29,   905,   537,  2846,    13,
           27,    14,    27, 

In [7]:
"""
    위의 예시는 정의된 AG_News를 활용했다면...
    일반적으로 Torch 사용시를 위해 Custom Dataset을 정의하고 DataLoader로 데이터를 불러와보자
    
    Custom Dataset 관련 - https://pytorch.org/tutorials/beginner/data_loading_tutorial.html 
    
    TextDataset -> 데이터 전처리 (tokenization, tensor만듬) / mapy-style dataset
    DataLoader의 collate_fn -> batch 단위의 Tensor 생성 
        maps style의 dataset을 사용했으므로 shuffling 등 가능
"""
from torch.utils.data import Dataset, DataLoader
import time

class TextDataset(Dataset):
    """
        custom dataset을 위해서 다음 3가지의 method를 구현해야함
        dataset은 크게 2종류로 만들 수 있음 (iterable style, map style)
        여기서는 map style로 만들어봄
    """
    def __init__(self, root,split, transform=None):
        
        train_iter = AG_NEWS(root=root,split=split) 
        self.train_data = [x for x in train_iter]
        self.transform = transform
        
    def __len__(self):
        return len(self.train_data)
    
    def __getitem__(self,idx):
        
#         print(idx)
#         print("\n")
        """ transform method를 통해 전처리 수행 후 tensor로 바꾸어서 Return """
        return self.transform(self.train_data[idx])

def transform(data):
    """
        tokenization 수행 후 tensor로 바꿈
    """
#     print("transform")
    get_token_id_list = vocab(tokenizer(data[1]))
    get_label_id = data[0]-1

    return torch.tensor(get_label_id), torch.tensor(get_token_id_list)
    

def collate_batch(batch):
    
    label_list, text_list, offsets = [], [],[0]

    for _label, _text in batch:
        label_list.append(_label)
        text_list.append(_text)
        offsets.append(_text.size(0))
    
    label_list=torch.tensor(label_list)
    text_list=torch.cat(text_list,0)
    offsets=torch.tensor(offsets).cumsum(dim=0)
    
    return label_list, text_list, offsets
    
text_dataset=TextDataset(root="./data/",split='train',transform=transform) 

""" 데이터가 잘 생성됨을 확인 """
# dataloader=DataLoader(text_dataset, batch_size=2, shuffle=True, collate_fn=collate_batch, )
# next(iter(dataloader)) 

""" Multi processing 사용시 데이터 전처리 후 Data Loading이 빨라짐을 확인 """


dataloader=DataLoader(text_dataset, batch_size=2, shuffle=False, collate_fn=collate_batch, num_workers=0, pin_memory=False)


print(next(iter(dataloader)))





(tensor([2, 2]), tensor([  431,   425,     1,  1605, 14838,   113,    66,     2,   848,    13,
           27,    14,    27,    15, 50725,     3,   431,   374,    16,     9,
        67507,     6, 52258,     3,    42,  4009,   783,   325,     1, 15874,
         1072,   854,  1310,  4250,    13,    27,    14,    27,    15,   929,
          797,   320, 15874,    98,     3, 27657,    28,     5,  4459,    11,
          564, 52790,     8, 80617,  2125,     7,     2,   525,   241,     3,
           28,  3890, 82814,  6574,    10,   206,   359,     6,     2,   126,
            1]), tensor([ 0, 29, 71]))
